In [1]:
import pandas as pd
import os
import numpy as np
from statistics import mean

In [17]:
papers_and_authors = pd.read_csv('E:/OtherCodeProjects/Springboard Capstone Projects/Springboard-Capstone-1-Data/papers_and_authors.csv')
len(papers_and_authors['paper_text'].value_counts())

7237

In [20]:
papers_and_authors[papers_and_authors.duplicated(keep=False)]

,year,title,event_type,pdf_name,abstract,paper_text,paper_id,author_id
14301,2015,Taming the Wild: A Unified Analysis of Hogwild...,Poster,5717-taming-the-wild-a-unified-analysis-of-hog...,Stochastic gradient descent (SGD) is a ubiquit...,Taming the Wild: A Unified Analysis of\nH OGWI...,5717,7222
14302,2015,Taming the Wild: A Unified Analysis of Hogwild...,Poster,5717-taming-the-wild-a-unified-analysis-of-hog...,Stochastic gradient descent (SGD) is a ubiquit...,Taming the Wild: A Unified Analysis of\nH OGWI...,5717,7222
14314,2015,Black-box optimization of noisy functions with...,Poster,5721-black-box-optimization-of-noisy-functions...,We study the problem of black-box optimization...,Black-box optimization of noisy functions with...,5721,6446
14315,2015,Black-box optimization of noisy functions with...,Poster,5721-black-box-optimization-of-noisy-functions...,We study the problem of black-box optimization...,Black-box optimization of noisy functions with...,5721,6446
14889,2015,Softstar: Heuristic-Guided Probabilistic Infer...,Poster,5889-softstar-heuristic-guided-probabilistic-i...,Recent machine learning methods for sequential...,Softstar: Heuristic-Guided Probabilistic Infer...,5889,6685
14890,2015,Softstar: Heuristic-Guided Probabilistic Infer...,Poster,5889-softstar-heuristic-guided-probabilistic-i...,Recent machine learning methods for sequential...,Softstar: Heuristic-Guided Probabilistic Infer...,5889,6685
15419,2015,Adaptive Low-Complexity Sequential Inference f...,Poster,6035-adaptive-low-complexity-sequential-infere...,We develop a sequential low-complexity inferen...,Adaptive Low-Complexity Sequential Inference f...,6035,8437
15420,2015,Adaptive Low-Complexity Sequential Inference f...,Poster,6035-adaptive-low-complexity-sequential-infere...,We develop a sequential low-complexity inferen...,Adaptive Low-Complexity Sequential Inference f...,6035,8437
15710,2016,Linear-Memory and Decomposition-Invariant Line...,Poster,6115-linear-memory-and-decomposition-invariant...,"Recently, several works have shown that natura...",Linear-Memory and Decomposition-Invariant\nLin...,6115,4889
15711,2016,Linear-Memory and Decomposition-Invariant Line...,Poster,6115-linear-memory-and-decomposition-invariant...,"Recently, several works have shown that natura...",Linear-Memory and Decomposition-Invariant\nLin...,6115,4889


In [3]:
auth_text_len = papers_and_authors.drop(['pdf_name', 'abstract', 'paper_id'], axis=1)
auth_text_len.head()

,year,title,event_type,paper_text,author_id
0,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1
1,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2
2,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14
3,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155
4,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54


In [4]:
print(len(auth_text_len.at[0, 'title']))

auth_text_len['title_len'] = auth_text_len.title.apply(lambda row: len(row))
auth_text_len.head()

62


,year,title,event_type,paper_text,author_id,title_len
0,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62
1,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62
2,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98
3,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116
4,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116


In [5]:
auth_text_len['paper_len'] = auth_text_len.paper_text.apply(lambda row: len(row))
auth_text_len.head()

,year,title,event_type,paper_text,author_id,title_len,paper_len
0,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62,21643
1,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62,21643
2,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98,15505
3,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116,20523
4,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116,20523


In [6]:
def avg_word_len(row):
    text_list = row.split()
    lengths = []
    for word in text_list:
        lengths.append(len(word))
    if len(lengths) > 0:
        return mean(lengths)
    else:
        return 0

In [7]:
avg_word_len(auth_text_len.paper_text[0])

4.8082635700783145

In [8]:
auth_text_len['avg_word_len'] = auth_text_len.paper_text.apply(lambda row: avg_word_len(row))
auth_text_len.head()

,year,title,event_type,paper_text,author_id,title_len,paper_len,avg_word_len
0,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62,21643,4.808264
1,1987,Self-Organization of Associative Database and ...,NaN,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62,21643,4.808264
2,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98,15505,4.886807
3,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116,20523,5.784861
4,1988,Storing Covariance by the Associative Long-Ter...,NaN,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116,20523,5.784861


In [9]:
print(auth_text_len.query('avg_word_len == 0'))

auth_text_len.drop(auth_text_len[auth_text_len['avg_word_len']==0].index, inplace=True)

print(auth_text_len.query('avg_word_len == 0'))

       year                                              title event_type  \
15924  2016  Regret Bounds for Non-decomposable Metrics wit...     Poster   
15925  2016  Regret Bounds for Non-decomposable Metrics wit...     Poster   
18546  2017  Mean Field Residual Networks: On the Edge of C...     Poster   
18547  2017  Mean Field Residual Networks: On the Edge of C...     Poster   

      paper_text  author_id  title_len  paper_len  avg_word_len  
15924         6597         62          9           0.0  
15925         3906         62          9           0.0  
18546         9809         50          9           0.0  
18547        10122         50          9           0.0  
Empty DataFrame
Columns: [year, title, event_type, paper_text, author_id, title_len, paper_len, avg_word_len]
Index: []


In [10]:
added_features = auth_text_len.fillna('Unknown')
added_features.drop(['title'], axis=1, inplace=True)
added_features.head()

,year,event_type,paper_text,author_id,title_len,paper_len,avg_word_len
0,1987,Unknown,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62,21643,4.808264
1,1987,Unknown,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62,21643,4.808264
2,1987,Unknown,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98,15505,4.886807
3,1988,Unknown,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116,20523,5.784861
4,1988,Unknown,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116,20523,5.784861


In [11]:
dummies = pd.get_dummies(added_features.event_type)
dummies.head()
added_dummies = added_features.join(dummies, how='outer')
added_dummies.drop(['event_type'], axis=1, inplace=True)
added_dummies.head()

,year,paper_text,author_id,title_len,paper_len,avg_word_len,Oral,Poster,Spotlight,Unknown
0,1987,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62,21643,4.808264,0,0,0,1
1,1987,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62,21643,4.808264,0,0,0,1
2,1987,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98,15505,4.886807,0,0,0,1
3,1988,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116,20523,5.784861,0,0,0,1
4,1988,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116,20523,5.784861,0,0,0,1


In [12]:
print(len(dummies))
print(len(added_features))

20839
20839


In [13]:
def jordan_detector(row):
    if row['author_id'] == 330:
        return 1
    else:
        return 0

In [14]:
added_dummies['is_jordan'] = added_dummies.apply(lambda row: jordan_detector(row), axis=1)
added_dummies.head()

,year,paper_text,author_id,title_len,paper_len,avg_word_len,Oral,Poster,Spotlight,Unknown,is_jordan
0,1987,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,1,62,21643,4.808264,0,0,0,1,0
1,1987,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,2,62,21643,4.808264,0,0,0,1,0
2,1987,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,14,98,15505,4.886807,0,0,0,1,0
3,1988,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,155,116,20523,5.784861,0,0,0,1,0
4,1988,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,54,116,20523,5.784861,0,0,0,1,0


In [15]:
df = added_dummies.drop('author_id', axis=1)
df.sort_values(by='is_jordan', ascending=False, inplace=True)
df.drop_duplicates(subset='paper_text', inplace=True)
len(df)

7236

In [16]:
len(df.query('is_jordan == 1'))

79